In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def sift(filename):
    # img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # 读取文件
    img = cv2.imread(filename)  # 读取文件
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    sift = cv2.SIFT.create()
    keyPoint, descriptor = sift.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor


def surf(filename):
    img = cv2.imread(filename)  # 读取文件
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    sift = cv2.xfeatures2d_SURF.create()
    keyPoint, descriptor = sift.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor


def orb(filename):
    # img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # 读取文件
    img = cv2.imread(filename)  # 读取文件
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    orb = cv2.ORB.create()
    keyPoint, descriptor = orb.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor


def stitch(filename1, filename2, filename3):
    img1, kp1, des1 = sift(filename1)
    img2, kp2, des2 = sift(filename2)
    img3, kp3, des3 = sift(filename3)
    bf = cv2.BFMatcher(cv2.NORM_L2)
    
    rawMatches1 = bf.knnMatch(des1, des2, 2)
    rawMatches2 = bf.knnMatch(des2, des3, 2)
    matches1 = []
    matches2 = []
    for m in rawMatches1:
        if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
            matches1.append((m[0].queryIdx, m[0].trainIdx))
    for m in rawMatches2:
        if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
            matches2.append((m[0].queryIdx, m[0].trainIdx))
    
    if len(matches1) > 4 and len(matches2) > 4:
        ptsA1 = np.float32([kp1[i].pt for (i, _) in matches1])
        ptsB1 = np.float32([kp2[i].pt for (_, i) in matches1])
        ptsA2 = np.float32([kp2[i].pt for (i, _) in matches2])
        ptsB2 = np.float32([kp3[i].pt for (_, i) in matches2])
        
        H1, _ = cv2.findHomography(ptsA1, ptsB1, cv2.RANSAC, 4.0)
        H2, _ = cv2.findHomography(ptsA2, ptsB2, cv2.RANSAC, 4.0)
        
        result1 = cv2.warpPerspective(img1, H1, (img1.shape[1] + img2.shape[1], img1.shape[0]))
        result1[0:img2.shape[0], 0:img2.shape[1]] = img2
        result2 = cv2.warpPerspective(result1, H2, (result1.shape[1] + img3.shape[1], result1.shape[0]))
        result2[0:img3.shape[0], 0:img3.shape[1]] = img3
        
        cv2.imshow('result', result2)
        cv2.waitKey(0)
        return result2
    
    return None

imageDir='images'
for item in os.listdir(imageDir):
    item_path = os.path.join(imageDir, item)
    if os.path.isdir(item_path):
        stitch(os.path.join(item_path,'1.jpg'), os.path.join(item_path,'2.jpg'), os.path.join(item_path,'3.jpg'))
